Module Imports

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

TensorFlow 2.x selected.


Mounting The Drive.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Downloading Fasttext Vectors

In [0]:
#Fasttext Vectors
downloaded = drive.CreateFile({'id':"10o7HhaaEDKp_O68_FOwho3nrVx_xuhh5"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cc.fa.300.vec.gz')        # replace the file name with your file

In [0]:
#Fasttext Vectors
downloaded = drive.CreateFile({'id':"1vQrV6RDI3e2tsHspMft6zM5DYvRK4wSD"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cc.en.300.vec.gz')        # replace the file name with your file

In [0]:
# importing required modules 
from zipfile import ZipFile 
  
import gzip
import shutil
with gzip.open('cc.fa.300.vec.gz', 'rb') as f_in:
    with open('file1.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [0]:
# importing required modules 
from zipfile import ZipFile 
  
import gzip
import shutil
with gzip.open('cc.en.300.vec.gz', 'rb') as f_in:
    with open('file2.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Create Embedding Index

In [0]:
import numpy as np 
embeddings_index_fa = {}
f = open('file1.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_fa[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_fa))

Found 2000000 word vectors.


In [0]:
import numpy as np 
embeddings_index_en = {}
f = open('file2.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_en[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_en))

Found 2000000 word vectors.


Downloading the Dataset

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"12wpQ2UBf6KolqO0yrWPArBf4hGmE65Ab"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Persian Many Things.xlsx')        # replace the file name with your file

Preprocessing the Data

In [0]:
#Reading the Dataset
import pandas as pd
english_data = []
persian_data = []

df=pd.read_excel(r'Persian Many Things.xlsx')
for i in range (df.shape[0]):
  english_data.append(df.iloc[i,0])
  persian_data.append(df.iloc[i,1])

In [0]:
#Clean Persian 
import re
def clean_fa(lines):
  punctuation = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟«؛«]+'
  en_pattern = "[a-zA-Z0-9]+"
  lines1=[]
  for line in lines:
    #create a space
    line = re.sub(r"([!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟«؛«'])", r" \1 ", line)
    line = re.sub(punctuation, ' ', line)
    line = re.sub(r'[" "]+', " ", line)
    lines1.append(line)
  return lines1

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
  
def clean_en(lines):
  lines1 = []
  for w in lines :
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z]+", " ", w)
    w = w.rstrip().strip()
    lines1.append(w)
  return lines1

In [0]:
pip install hazm

     |████████████████████████████████| 317kB 3.4MB/s 
     |████████████████████████████████| 1.4MB 62.9MB/s 
     |████████████████████████████████| 235kB 34kB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394473 sha256=603825f500d9bc2766267cdb9083b0d3d69537d1ef9f28672ae982fdf3da8005
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154649 sha256=0f918881bcc1a158e7e436f880050611f354273e013b4b078bc67544428280fa
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
from hazm import sent_tokenize, word_tokenize

In [0]:
totalX = []
for i in range(len(persian_data)):
  text=str(persian_data[i])
  seg_list = word_tokenize(text)
  text = " ".join(seg_list)
  totalX.append(text)

In [0]:
en_clean=clean_en(english_data)
fa_clean=clean_fa(totalX)
english_clean=[]
persian_clean=[]
for i in range(len(en_clean)):#was en_clean
  en_clean[i] = '<start> ' + str(en_clean[i]) + ' <end>'
  fa_clean[i] = '<start> ' + str(fa_clean[i]) + ' <end>'
  english_clean.append(en_clean[i])
  persian_clean.append(fa_clean[i])

In [0]:
def convert_list(list):
  return tuple(list)

In [0]:
en=convert_list(english_clean)
fa=convert_list(persian_clean)

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def tensor_and_tokeniser(inp_lang, targ_lang):
  # creating cleaned input, output pairs
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = tensor_and_tokeniser(fa, en)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [0]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

16963 16963 4241 4241


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

Create a tf.data dataset

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 85
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

Creating the Model Architecture

In [0]:
#creating input embedding matrix
embedding_dim = 300
embedding_matrix_fa = np.zeros((vocab_inp_size, embedding_dim))
for word, i in inp_lang.word_index.items() :
    embedding_vector = embeddings_index_fa.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_fa[i] = embedding_vector

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    # self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix_fa],
                            input_length=None,
                            trainable=False, mask_zero = True)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)

In [0]:
#creating output embedding matrix 
embedding_matrix_en = np.zeros((vocab_tar_size, embedding_dim))
for word, i in targ_lang.word_index.items() :
    embedding_vector = embeddings_index_en.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_en[i] = embedding_vector

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    # self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.embedding = tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix_en],
                            input_length=None,
                            trainable=False, mask_zero = True)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

Optimizer and the Loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

Training

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 1 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.2099
Epoch 1 Batch 100 Loss 0.1953
Epoch 1 Loss 0.2043
Time taken for 1 epoch 382.5240201950073 sec

Epoch 2 Batch 0 Loss 0.1628
Epoch 2 Batch 100 Loss 0.1620
Epoch 2 Loss 0.1662
Time taken for 1 epoch 378.7786068916321 sec

Epoch 3 Batch 0 Loss 0.1040


KeyboardInterrupt: ignored

Inference

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  punctuation = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟«؛«]+'
  en_pattern = "[a-zA-Z0-9]+"
  seg_list = word_tokenize(sentence)
  sentence = " ".join(seg_list)
  sentence = re.sub(r"([!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟«؛«'])", r" \1 ", sentence)
  sentence = re.sub(punctuation, ' ', sentence)
  sentence = '<start> ' + str(sentence) + ' <end>'
  sentence = re.sub(r'[" "]+', " ", sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    # attention_weights = tf.reshape(attention_weights, (-1, ))
    # attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

Restoring the model

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

Evaluate 

In [0]:
translate('او یک دختر است')

Input: <start> او یک دختر است <end>
Predicted translation: he s a daughter <end> 


In [0]:
translate('او بعد از شام تحصیل می کند')

Input: <start> او بعد از شام تحصیل می کند <end>
Predicted translation: he studies after dinner <end> 


In [0]:
translate('او در مدرسه تحصیل می کند')

Input: <start> او در مدرسه تحصیل می کند <end>
Predicted translation: he is studying school school <end> 


In [0]:
translate('ارتش روزها جنگید')

Input: <start> ارتش روزها جنگید <end>
Predicted translation: the enemy war <end> 


In [0]:
translate('سربازان سالها می جنگیدند')

Input: <start> سربازان سالها می جنگیدند <end>
Predicted translation: the children were almost <end> 


In [0]:
translate('اجرا کن')

Input: <start> اجرا کن <end>
Predicted translation: keep on <end> 


In [0]:
translate('منطقه می‌شود.')

KeyError: ignored